In [ ]:
import pickle

with open('/filepath/lhs_exercises_trans.txt', 'rb') as read:
  ex_trans = pickle.load(read)
with open('/filepath/lhs_exercises_nontrans.txt', 'rb') as read:
  ex_nontrans = pickle.load(read)

print(len(ex_trans))
print(len(ex_nontrans))

In [ ]:
#Chordal transformation

demo = [([60, 0.5], 1), ([[60, 64], 0.5], [1,2]), ([[60, 64, 67], 0.5], [1,2,3]), ([[60, 64, 67, 70], 1.5], [1,2,3,4])]

def chord_to_seq(exercise):
  seq_exercise = []
  for note in exercise:
    if type(note[0][0]) == int:
      seq_exercise.append(note)
    elif type(note[0][0]) == list:
      chordsize = len(note[0][0])
      for count, pitch in enumerate(note[0][0]):
          if count < chordsize - 1:
            seq_exercise.append(([pitch, 0], note[1][count]))
          else:
            seq_exercise.append(([pitch, note[0][1]], note[1][count]))
    else:
      print("Type error")
      break
  return seq_exercise

demo_cts = chord_to_seq(demo)
print(demo_cts)

In [ ]:
#Transposition for the transposable exercises
#Modified to transpose to every pitch and range of the instrument. Midi note 36 is now 0 and Midi note 96 is now 60
demo_cts = [([60, 0.5], 1), ([60, 0], 1), ([64, 0.5], 2), ([60, 0], 1), ([64, 0], 2), ([67, 0.5], 3), ([60, 0], 1), ([64, 0], 2), ([67, 0], 3), ([70, 1.5], 4)]

def transposition(exercise):

  idx_list = []
  for note_data in exercise:
    idx_list.append(note_data[0][0])
  min_pitch = min(idx_list)
  max_pitch = max(idx_list)
  exercise_width = max_pitch - min_pitch
  num_transpositions = 61 - exercise_width

  transposed = []
  for i in range(num_transpositions):
    ex_trans = []
    for note in exercise:
      ex_trans.append(([note[0][0]-min_pitch+i, note[0][1]], note[1])) #note - now we care if notes go beyond the range of the marimba because it WILL matter once we apply one-hot pitch encoding
    transposed.append(ex_trans)
  return transposed

demo_cts_trans = transposition(demo_cts)
print(demo_cts_trans)

In [ ]:
#Transposition for the nontransposable exercises - that is, to copy the non-transposable exercises into each octave of the marimba
#Modified to transpose to every octave of the instrument. Midi note 36 is now 0 and Midi note 96 is now 60
demo_cts = [([60, 0.5], 1), ([60, 0], 1), ([64, 0.5], 2), ([60, 0], 1), ([64, 0], 2), ([67, 0.5], 3), ([60, 0], 1), ([64, 0], 2), ([67, 0], 3), ([70, 1.5], 4)]

def nontrans_transposition(exercise):

  idx_list = []
  for note_data in exercise:
    idx_list.append(note_data[0][0])
  min_pitch = min(idx_list)
  max_pitch = max(idx_list)
  octaves_down = min_pitch // 12
  num_transpositions = 1 + (61 - (max_pitch - (12*octaves_down))) // 12

  transposed = []
  for i in range(num_transpositions):
    ex_trans = []
    for note in exercise:
      ex_trans.append(([note[0][0] + (12*(i-octaves_down)), note[0][1]], note[1])) #note - now we care if notes go beyond the range of the marimba because it WILL matter once we apply one-hot pitch encoding
    transposed.append(ex_trans)
  return transposed

demo_cts_trans = nontrans_transposition(demo_cts)
print(demo_cts_trans)

In [ ]:
#One-hot encoding of sticking input and output

demo_cts_trans_latt_0_intfwd = [([0, 0.5], 1), ([0, 0], 1), ([4, 0.5], 2), ([0, 0], 1), ([4, 0], 2), ([7, 0.5], 3), ([0, 0], 1), ([4, 0], 2), ([7, 0], 3), ([10, 1.5], 4)]

def onehot_sticking(exercise):
  ex_onehot = []
  for note in exercise:
    input_vector = [0] * 62
    input_vector[note[0][0]] = 1
    input_vector[61] = note[0][1]

    stick_vector = [0,0,0,0]
    stick_vector[note[1]-1] = 1

    ex_onehot.append((input_vector, stick_vector))
  return ex_onehot

print(onehot_sticking(demo_cts_trans_latt_0_intfwd))
    

In [ ]:
ex_trans_prep = []
for exercise in ex_trans:
  for trans_exercise in transposition(chord_to_seq(exercise)):
    ex_trans_prep.append(onehot_sticking(trans_exercise))

ex_nontrans_prep = []
for exercise in ex_nontrans:
  for nontrans_exercise in nontrans_transposition(chord_to_seq(exercise)):
    ex_nontrans_prep.append(onehot_sticking(chord_to_seq(nontrans_exercise)))

lhs_data_old_assumptions = ex_trans_prep + ex_nontrans_prep


with open('/filepath/lhs_dataset_old_assumptions_prepped.txt', 'wb') as f:
  pickle.dump(lhs_data_old_assumptions, f)